# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [ ]:
! rm -r /kaggle/working/*

In [ ]:
from IPython.display import Markdown

# Load the tenders

Locate the tenders - in PDF format.

In [ ]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

In [ ]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

In [ ]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

Installing required Python packages to analyze the tender - in PDF format.

In [ ]:
!pip install PyPDF2

Extract information from the tenders.
The output will be a text.

In [ ]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = "\n".join([page.extract_text() for page in reader.pages])

# information for each tender can be accessed by:
# tenders_info[tenders[0]]

# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [ ]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

Locate the JSON file containing the companies' information.

In [ ]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

Define a small function to read the information about the companies - in JSON format.

In [ ]:
import json

def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [ ]:
companies_info = read_json_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [ ]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

## Analyze all the tenders

In [ ]:
download_tenders_json_from_repo = True # switch to False if willing to generate the json file within this notebook

if download_tenders_json_from_repo:
    # this helps reducing the Gemini Quota, since no queries will be performed for the tenders
    github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder=tenders_file_path, extension="json")

In [ ]:
tender_prompt_template = "The document you have is a tender, that contains technical requirements for a project. Summarize the technical requirements. The content of the document is: "
tender_prompts = []
for info in tenders_info.values():
    tender_prompts.append(f"You have a document called {info['name']} . " + tender_prompt_template + f"{info['content']}")

In [ ]:
from time import sleep

responses = {}
tenders_json_file_path = os.path.join(tenders_file_path, 'tenders.json')
if os.path.exists(tenders_json_file_path):
    responses = read_json_info(tenders_json_file_path)
    print(f"Responses loaded from file {tenders_json_file_path}")
else:
    num_queries = 0
    for tender_prompt, tender_name in zip(tender_prompts, tenders):
        print(f"Generating response for tender {tender_name} ...")
        response = chat.send_message(tender_prompt)
        # print(response.text)
        responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
        print(f"Response for tender {tender_name} generated.")

        num_queries += 1
        """
        if num_queries % 2 == 0:
            wait_time_seconds = 90
            print(f"Waiting {wait_time_seconds} before continuing, to not exceed Gemini's quota")
            sleep(wait_time_seconds)
        """

    with open(tenders_json_file_path, 'w') as f:
        json.dump(responses, f, ensure_ascii=True, indent=4)
    print(f"Responses stored into {tenders_json_file_path}")

print("Analysis of the tenders concluded!")

In [ ]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []
for idx_response, response in enumerate(responses.values()):
    query = Part()
    query.text = response['prompt']
    # TODO: consider different users? Example: for the tender number idx_response, use f"user_{idx_response}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))

## Define the system and user prompts

In [ ]:
system_prompt = "You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you."

Markdown(system_prompt)

In [ ]:
user_prompt = f"""

1. Identify all the technical requirements in the tenders' information you have.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results.

3. Calculate an affinity score in percentage of company [SIEMENS] and [HITACHI] based on the match of results in point 2. Explain the way how you computed this percentage.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# Markdown(user_prompt)

## Test the history

In [ ]:
chat_with_memory_temp = model.start_chat(history=history_chat)
response_temp = chat_with_memory_temp.send_message("What are the tenders you have about?")

Markdown(response_temp.text)

## Count the needed tokens

In [ ]:
print(f"{model.count_tokens(history_chat)=}")
print(f"{model.count_tokens(system_prompt + user_prompt)=}")

## Generate a response with chat history

In [ ]:
chat_with_memory = model.start_chat(history=history_chat)

print("Providing prompts based on the previous analysis of the tenders ...")
response = chat_with_memory.send_message(f"{system_prompt + user_prompt}")
# print(response.text)
responses['main_query'] = {'prompt': system_prompt + user_prompt, 'answer': response.text}

print("Response to the prompts is ready!")

In [ ]:
Markdown(responses['main_query']['answer'])